In [2]:
import numpy as np
import pandas as pd
from pycirclize import Circos

In [3]:
df = pd.read_excel('../data/br2f_topographic_data.xlsx', header=0, index_col=0)

In [4]:
# define left hemisphere
brain_regions = df.index.to_numpy()

left_hemisphere = brain_regions[:392]
df_left_hemisphere = df[left_hemisphere]
df_left_hemisphere = df_left_hemisphere.head(392)

## Hippocampal - Septostri connectivity plots

Next we will look at one aggregated region (LS) and three non-aggregated regions (MS, NDB, ACB) connectivity with the hippocampus. Each of the four regions is an aggregation across other subregions, so we start by summing across the subregions.

In [5]:
# construct regions
LS_aggregated_regions = np.array(['LSr.m.v','LSr.m.d','LSr.vl','LSr.dl','LSc.v','LSc.d','LSv'])

LS_non_aggregated_regions = np.array(['MS','NDB','ACB'])

hippocampus_regions = np.array(['DG','CA3','CA2','CA1v','CA1d','SUBv','SUBd','PAR', 'PRE', 'POST'])

septostri_regions = np.concatenate(
    (LS_aggregated_regions, LS_non_aggregated_regions), axis=0
)

spt_hipp_regions = np.concatenate((septostri_regions, hippocampus_regions), axis=0)

In [6]:
# reduce the left hemisphere dataframe to only include hypothalamus and hippocampal regions
df_spt_hipp = df_left_hemisphere[spt_hipp_regions]
df_spt_hipp = df_spt_hipp[df_spt_hipp.index.isin(spt_hipp_regions)]

In [14]:
#LHA
# rows
LS_rows = df_spt_hipp[df_spt_hipp.index.isin(LS_aggregated_regions)].sum(axis=0)
df_spt_hipp.loc["LS"] = LS_rows

# delete extra LHA subregions from rows
df_spt_hipp = df_spt_hipp.loc[
~df_spt_hipp.index.isin(LS_aggregated_regions)
]

In [15]:
# columns
LS_col_sum = df_spt_hipp[LS_aggregated_regions].sum(axis=1)
df_spt_hipp["LS"] = LS_col_sum

# delete extra LHA subregion from columns
df_spt_hipp = df_spt_hipp.drop(LS_aggregated_regions, axis=1)

/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_25455/1210777283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spt_hipp["LS"] = LS_col_sum


In [16]:
df_spt_hipp

,MS,NDB,ACB,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LS
Abbr.,,,,,,,,,,,,,,
MS,0,5,2,5,3,2,3,2,5,3,4,2,1,12
NDB,2,0,0,3,3,1,1,4,2,4,6,1,4,10
ACB,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,0,6,0,0,0,0,0,0,0,0,0
CA3,2,2,0,3,0,6,6,6,3,3,0,0,0,19
CA2,0,0,0,4,4,0,4,0,3,0,2,0,0,9
CA1v,3,3,5,2,1,2,0,0,7,0,4,2,0,20
CA1d,0,1,1,2,0,0,0,0,0,7,1,2,0,13
SUBv,0,0,6,3,3,3,3,0,0,0,5,4,0,27


In [17]:
# remove self-connections (set diagonal to 0)
for i in range(len(df_spt_hipp.index)):
    df_spt_hipp.iloc[i,i] = 0

In [18]:
# we need to remove connections within the hypothalamus--
# we're only interested in hippocampal-hypothalamus connectivity

septostri_aggregated_regions = np.array(['LS','MS','NDB','ACB'])
for subregion_i in range(len(septostri_aggregated_regions)):
    for subregion_j in range(len(septostri_aggregated_regions)):
        df_spt_hipp.loc[
        septostri_aggregated_regions[subregion_i], 
        septostri_aggregated_regions[subregion_j]
        ] = 0

In [19]:
df_spt_hipp

,MS,NDB,ACB,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LS
Abbr.,,,,,,,,,,,,,,
MS,0,0,0,5,3,2,3,2,5,3,4,2,1,0
NDB,0,0,0,3,3,1,1,4,2,4,6,1,4,0
ACB,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,0,6,0,0,0,0,0,0,0,0,0
CA3,2,2,0,3,0,6,6,6,3,3,0,0,0,19
CA2,0,0,0,4,4,0,4,0,3,0,2,0,0,9
CA1v,3,3,5,2,1,2,0,0,7,0,4,2,0,20
CA1d,0,1,1,2,0,0,0,0,0,7,1,2,0,13
SUBv,0,0,6,3,3,3,3,0,0,0,5,4,0,27


In [21]:
#clear out intrinsic connections between hippocampus_regions
for subregion_i in range(len(hippocampus_regions)):
    for subregion_j in range(len(hippocampus_regions)):
        df_spt_hipp.loc[
        hippocampus_regions[subregion_i], 
        hippocampus_regions[subregion_j]
        ] = 0

In [23]:
df_spt_hipp

,MS,NDB,ACB,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LS
Abbr.,,,,,,,,,,,,,,
MS,0,0,0,5,3,2,3,2,5,3,4,2,1,0
NDB,0,0,0,3,3,1,1,4,2,4,6,1,4,0
ACB,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CA3,2,2,0,0,0,0,0,0,0,0,0,0,0,19
CA2,0,0,0,0,0,0,0,0,0,0,0,0,0,9
CA1v,3,3,5,0,0,0,0,0,0,0,0,0,0,20
CA1d,0,1,1,0,0,0,0,0,0,0,0,0,0,13
SUBv,0,0,6,0,0,0,0,0,0,0,0,0,0,27


In [43]:
# delete rows with all 0's
df_spt_hipp_cleaned = df_spt_hipp.loc[(df_spt_hipp.T!=0).any(),:]

In [44]:
df_spt_hipp_cleaned

,MS,NDB,ACB,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LS
Abbr.,,,,,,,,,,,,,,
MS,0,0,0,5,3,2,3,2,5,3,4,2,1,0
NDB,0,0,0,3,3,1,1,4,2,4,6,1,4,0
CA3,2,2,0,0,0,0,0,0,0,0,0,0,0,19
CA2,0,0,0,0,0,0,0,0,0,0,0,0,0,9
CA1v,3,3,5,0,0,0,0,0,0,0,0,0,0,20
CA1d,0,1,1,0,0,0,0,0,0,0,0,0,0,13
SUBv,0,0,6,0,0,0,0,0,0,0,0,0,0,27
SUBd,2,2,4,0,0,0,0,0,0,0,0,0,0,16
LS,0,0,0,1,6,1,3,2,2,0,0,0,0,0


In [49]:
# re-order rows for more coherent representation in plot

df_spt_hipp_cleaned = df_spt_hipp_cleaned.loc[
['CA1v','CA1d','CA2','CA3','SUBv','SUBd','MS','NDB','LS'],
['CA1v','CA1d','CA2','CA3','DG','PAR','PRE','POST','SUBv','SUBd','MS','NDB','ACB','LS']
]

In [50]:
df_spt_hipp_cleaned

,CA1v,CA1d,CA2,CA3,DG,PAR,PRE,POST,SUBv,SUBd,MS,NDB,ACB,LS
Abbr.,,,,,,,,,,,,,,
CA1v,0,0,0,0,0,0,0,0,0,0,3,3,5,20
CA1d,0,0,0,0,0,0,0,0,0,0,0,1,1,13
CA2,0,0,0,0,0,0,0,0,0,0,0,0,0,9
CA3,0,0,0,0,0,0,0,0,0,0,2,2,0,19
SUBv,0,0,0,0,0,0,0,0,0,0,0,0,6,27
SUBd,0,0,0,0,0,0,0,0,0,0,2,2,4,16
MS,3,2,2,3,5,4,2,1,5,3,0,0,0,0
NDB,1,4,1,3,3,6,1,4,2,4,0,0,0,0
LS,3,2,1,6,1,0,0,0,2,0,0,0,0,0


In [53]:
circos_hyp_hipp_extrinsic_only = Circos.initialize_from_matrix(df_spt_hipp_cleaned, 
                                                               space=5,
                                                               cmap="jet")

circos_hyp_hipp_extrinsic_only.savefig("../output/hippocampus_septostri_extrinsic_connectivity.svg")